**Дисклеймер:** весь текст этой тетрадки переведу на английский после успешной реализации.

In [1]:
import pandas as pd

In [2]:
fintech = pd.read_csv('fintech.csv')
industrial_goods = pd.read_csv('industrial_goods.csv')
renewable_energy = pd.read_csv('renewable_energy.csv')
healthcare_services = pd.read_csv('healthcare_services.csv')

## Description of the variables

## Processing of Nans/passes

In [3]:
print(f'Initial (total) amount of NaNs')
print()
print(f'In fintech data: {fintech.isna().sum().sum()}')
print(f'In industrial goods data: {industrial_goods.isna().sum().sum()}')
print(f'In renewable energy data: {renewable_energy.isna().sum().sum()}')
print(f'In healthcare services data: {healthcare_services.isna().sum().sum()}')

Initial (total) amount of NaNs

In fintech data: 208642
In industrial goods data: 209596
In renewable energy data: 94342
In healthcare services data: 178072


Сайт FMP позволяет парсить котировки американских акций только за последние 5 лет. В связи с этим создадим вспомогательные переменные

In [4]:
the_day_of_parsing = pd.to_datetime('2024-05-08', format='%Y-%m-%d')
threshold = the_day_of_parsing - pd.DateOffset(years=5)

### Fintech

Надеюсь, можно будет впоследствии запихнуть это в функцию и экстраполировать на все отрасли

In [5]:
fintech['date'] = pd.to_datetime(fintech['date'], format='%Y-%m-%d')
fintech.shape

(1976, 293)

In [6]:
fintech.isna().sum().sort_values(ascending=False)

close_MBNK_RU          1968
volume_MBNK_RU         1968
close_CARM_RU          1758
volume_CARM_RU         1758
volume_CRBG_USA        1562
                       ... 
Денежный агрегат М0      29
Денежный агрегат М1      29
Денежный агрегат М2      29
effr                     29
date                      0
Length: 293, dtype: int64

**Первый шаг обработки:** для удобства определения причины пропусков создадим столбец-признак `is_workday`, который принимает два значения: True в будни и False в выходные/праздники. Для этого обратимся к данным из календаря, который мы спарсили ранее.

In [7]:
calendar = pd.read_csv('data/trading_calendat_NYSE.csv')
calendar.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
calendar['Date'] = pd.to_datetime(calendar['Date'], format='%Y-%m-%d')
calendar['Date'].dtype

dtype('<M8[ns]')

**Первый шаг исследования:** посмотрев на таблицу, мы поняли, что надо снова отсечь данные по дате на 5 лет ранее момента парсинга, потому что FMP дает данные о котировках только за последние 5 лет.

In [9]:
#дату (месяц/день) надо менять в зависимости от момента парсинга!!

# ВТОРАЯ СТРОЧКА В ЭТОМ БЛОКЕ ПОКА НЕ РАБОТАЕТ (причина не ясна, результат плачевный)
work_calendar = calendar[calendar['Date'] > threshold]
fintech['is_workday'] = [i in work_calendar['Date'].dt.date for i in fintech['date'].dt.date]
fintech = fintech[fintech['date'] > threshold]

In [10]:
work_calendar['Date']

1346   2019-05-09
1347   2019-05-10
1348   2019-05-13
1349   2019-05-14
1350   2019-05-15
          ...    
2598   2024-04-30
2599   2024-05-01
2600   2024-05-02
2601   2024-05-03
2602   2024-05-06
Name: Date, Length: 1257, dtype: datetime64[ns]

**ПАЧИМУ((**

In [11]:
{i in work_calendar['Date'] for i in fintech['date']}

{False}

Для удобства исследования создадим новый столбец: день недели

In [12]:
fintech['day_of_week'] = fintech['date'].dt.day_name()

In [13]:
fintech.set_index(['date'], inplace=True)
fintech.sort_index(inplace=True)

In [14]:
fintech.head(4)

,volume_NTRS_USA,close_NTRS_USA,volume_EWBC_USA,close_EWBC_USA,volume_COIN_USA,close_COIN_USA,volume_CG_USA,close_CG_USA,volume_CARM_RU,close_CARM_RU,...,Low,Close,Volume,Interbank credit market rates,gold_rates,silver_rates,platinum_rates,palladium_rates,is_workday,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2019-05-09,804008.0,96.97,748222.0,50.17,NaN,NaN,593700.0,20.76,NaN,NaN,...,NaN,NaN,NaN,6.9477,3290.00,41.27,2069.61,3303.26,False,Thursday
2019-05-10,731618.0,97.56,674296.0,50.19,NaN,NaN,405600.0,20.62,NaN,NaN,...,2511.16,2514.87,3.253956e+10,NaN,3156.00,36.81,1848.23,3472.75,False,Friday
2019-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.3560,NaN,NaN,NaN,NaN,False,Saturday
2019-05-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.3851,3046.02,35.11,1874.03,3836.81,False,Sunday


**Наблюдение:** после парсинга данных о `S&P500` и `imoex` от каждого появилось по 5 показателей (`high`, `low`, `open`, `close`, `volume`). Мы решили, что может быть польза именно от гэпов `high-low` и `close-open`, поэтому 5 показателей от каждого индекса можно заменить на 4: `volume`, `gap_hl = high-low`, `gap_co = close-open` и `close`.

In [15]:
fintech['S&P500_gap_hl'] = fintech['high'] - fintech['low']
fintech['S&P500_gap_co'] = fintech['close'] - fintech['open']
fintech['IMOEX_gap_hl'] = fintech['High'] - fintech['Low']
fintech['IMOEX_gap_co'] = fintech['Close'] - fintech['Open']

In [16]:
fintech.drop(['High', 'Low', 'high', 'low', 'open', 'Open'], axis=1, inplace=True)

Переименуем некоторые (схожие по названию) столбцы, чтобы не было путаницы

In [17]:
fintech.rename({'Close': 'IMOEX_close', 'close': 'S&P500_close', 'Volume': 'IMOEX_volume', 'volume': 'S&P500_volume'}, axis=1, inplace=True)

In [18]:
fintech[fintech['IMOEX_volume'].isnull()].sample(5)

,volume_NTRS_USA,close_NTRS_USA,volume_EWBC_USA,close_EWBC_USA,volume_COIN_USA,close_COIN_USA,volume_CG_USA,close_CG_USA,volume_CARM_RU,close_CARM_RU,...,gold_rates,silver_rates,platinum_rates,palladium_rates,is_workday,day_of_week,S&P500_gap_hl,S&P500_gap_co,IMOEX_gap_hl,IMOEX_gap_co
date,,,,,,,,,,,,,,,,,,,,,
2019-05-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2662.03,30.22,1674.29,2771.09,False,Saturday,NaN,NaN,NaN,NaN
2023-06-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,Sunday,NaN,NaN,NaN,NaN
2021-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4310.03,63.07,2933.22,6885.23,False,Saturday,NaN,NaN,NaN,NaN
2021-02-23,931936.0,97.55,1008525.0,71.35,NaN,NaN,2580600.0,36.23,NaN,NaN,...,NaN,NaN,NaN,NaN,False,Tuesday,8.745,2.84,NaN,NaN
2021-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,Sunday,NaN,NaN,NaN,NaN


**Наблюдение:** большинство данных не спарсилось именно за выходные. это объясняется тем, что (почти) все отдыхают. Про праздники не будет сразу делать таких выводов, потому что там все зависит от организации.

**Решение:** избавимся от этих данных, с ними кашу не сваришь.

In [19]:
fintech = fintech[(fintech['day_of_week'] != 'Sunday') & (fintech['day_of_week'] != 'Saturday')]

In [20]:
fintech.isna().sum().sum()

39657

Также надо не забыть, что если парсинг американских акций был не сегодня, то надо отсечь данные по значению переменной `the_day_of_parsing`.

In [21]:
fintech = fintech[fintech.index <= the_day_of_parsing]

In [22]:
fintech.isna().sum().sum()

35105

## Исследование данных о котировках акций

Теперь рассмотрим каждый признак на предмет наличия пропусков детальнее:

In [23]:
fintech.isna().sum().sort_values(ascending=False)

volume_MBNK_RU         1298
close_MBNK_RU          1298
volume_CARM_RU         1088
close_CARM_RU          1088
volume_CRBG_USA         891
                       ... 
Денежный агрегат М1       6
Денежный агрегат М2       6
effr                      6
is_workday                0
day_of_week               0
Length: 292, dtype: int64

**Замечание:** заметим, что по акциям некоторых компаний практически нет данных. Так случилось из-за того, что конкретно те сайты, откуда мы доставали котировки, не хранили их по этим компаниям, может, компании слишком непопулярны, либо их акции начали торговаться гораздо позже 2019 года и тд

**Со многими** акциями подобных компаний мы попрощаемся в рамках нашего исследования. Для удобства резделим данные на те, где пропусков больше 500 и где пропусков от 150 до 500 (этот промежуток требует более пристального рассмотрения, потому что они имеют больший шанс остаться с ручным заполнением пропусков).

In [24]:
exceptions = list(fintech.isna().sum()[fintech.isna().sum() < 500].index)
fintech_1 = fintech.loc[:, ~fintech.columns.isin(exceptions)]
fintech_1.isna().sum().sort_values(ascending=False)

volume_MBNK_RU     1298
close_MBNK_RU      1298
volume_CARM_RU     1088
close_CARM_RU      1088
close_CRBG_USA      891
volume_CRBG_USA     891
close_TPG_USA       723
volume_TPG_USA      723
close_POSI_RU       722
volume_POSI_RU      702
close_SPBE_RU       702
volume_NU_USA       699
close_NU_USA        699
volume_SPBE_RU      682
close_RENI_RU       682
volume_RENI_RU      661
volume_HOOD_USA     606
close_HOOD_USA      606
close_RYAN_USA      601
volume_RYAN_USA     601
close_COIN_USA      532
volume_COIN_USA     532
dtype: int64

In [25]:
many_nans_1 = list(fintech_1.isna().sum().sort_values(ascending=False).index)

Исследуем некоторые акции по отдельности: почему у них так много пропусков?

**Описание процесса:** предполагаемых причин начилия большого количества пропусков было несколько: либо компания появилась на финансовом рынке совсем недавно (относительно промежутка 2019-2024 год), либо, в случае акций российских компаний, они торговались в разных режимах (не только TQBR) - предположение на уровне гипотезы, либо конкретно тот сайт, откуда мы брали данные о котировках, не предоставлял данные за какой-то период.

Как оказалось, **причина большинства пропусков** - компания разметила акции на финансовом рынке не так давно. **Причина дисбалансов (у volume/close)** - где-то вместо пропусков спарсились нули.

In [26]:
exceptions_1 = list(fintech.isna().sum()[(fintech.isna().sum() > 500) | (fintech.isna().sum() < 150)].index)
fintech_2 = fintech.loc[:, ~fintech.columns.isin(exceptions_1)]

In [27]:
fintech_2.isna().sum().sort_values(ascending=False).drop(['usdrub', 'palladium_rates', 'platinum_rates', 'silver_rates', 'gold_rates', 'Interbank credit market rates'])

volume_OWL_USA     450
close_OWL_USA      450
close_SMLT_RU      434
volume_SMLT_RU     414
volume_RKT_USA     360
close_RKT_USA      360
close_INGR_RU      286
volume_UWMC_USA    282
close_UWMC_USA     282
volume_INGR_RU     266
close_PRMB_RU      226
close_RDRB_RU      219
close_TCSG_RU      203
volume_XP_USA      196
close_XP_USA       196
volume_TCSG_RU     163
dtype: int64

**Промежуточный вывод:** исследовав все акции, где количество пропусков превосходит 10%, можно сказать, что почти у всех причина одна: поздний листинг на фондовом рынке. Исключениями являются `INGR` и `PRMB`: в случае `INGR` есть подозрение, что данные неправильно спарсились, либо мосбиржа долго не публиковала их; у `PRMB` пропуски идут не подряд, а эпизодически, из-за чего их можно попробовать заполнить, предварительно предсказав.

In [28]:
many_nans_2 = list(fintech_2.isna().sum().sort_values(ascending=False).drop(['usdrub', 'palladium_rates', 'platinum_rates', 'silver_rates', 'gold_rates', 'Interbank credit market rates', 'close_PRMB_RU']).index)

Котировки акций компании `INGR` мы решили выкинуть, потому что парсинг был реализован правильно и этот случай - странное исключение, которое сложно обработать.

**Итого:** избавляемся от данных о 19 акциях (`MBNK`, `CARM`, `CRBG`, `TPG`, `POSI`, `SPBE`, `NU`, `RENI`, `HOOD`, `RYAN`, `COIN`, `OWL`, `SMLT`, `RKT`, `INGR`, `UWMC`, `RDRB`, `TCSG`, `XP`)

In [29]:
all_columns_to_drop = many_nans_1 + many_nans_2
fintech.drop(all_columns_to_drop, axis=1, inplace=True)

In [30]:
fintech.shape

(1305, 255)

In [31]:
fintech.isna().sum().sort_values(ascending=False)

usdrub                    371
palladium_rates           371
platinum_rates            371
silver_rates              371
gold_rates                371
                         ... 
Денежный агрегат М2         6
Широкая денежная масса      6
effr                        6
is_workday                  0
day_of_week                 0
Length: 255, dtype: int64

In [32]:
list_of_features = fintech.isna().sum().sort_values(ascending=False).index
filtered_columns = [col for col in list_of_features if not (col.startswith('volume_') or col.startswith('close_') or (fintech[col].isna().sum() == 0))]
filtered_columns

['usdrub',
 'palladium_rates',
 'platinum_rates',
 'silver_rates',
 'gold_rates',
 'Interbank credit market rates',
 'daily_gdp_no_season',
 'daily_gdp',
 'IMOEX_close',
 'IMOEX_volume',
 'IMOEX_gap_hl',
 'IMOEX_gap_co',
 'S&P500_close',
 'S&P500_gap_co',
 'S&P500_gap_hl',
 'S&P500_volume',
 ' Долг нефинансового сектора и домашних хозяйств, итого',
 'Кредиты кредитных организаций',
 'Котируемые акции и паи и акции инвестиционных фондов',
 'Долговые ценные бумаги',
 'Денежные средства на брокерских счетах',
 'Депозиты',
 'Наличная иностранная валюта',
 '    Домашние хозяйства и НКООДХ',
 '    Нефинансовый сектор',
 '          Внутренние заимствования',
 '              Кредиты',
 '              Долговые ценные бумаги в портфеле резидентов',
 '          Внешние заимствования',
 '              Кредиты и займы, полученные от нерезидентов',
 'Наличная национальная валюта',
 '              Долговые ценные бумаги в портфеле нерезидентов, включая векселя',
 '          Внутренние заимствования.1

**Глобально** мы будем пользоваться двумя способами заполнения пропусков: наивное предсказание через `.ffill()` для признаков, на которые потенциально смотрят инвесторы для принятия решения. Делаем так, потому что они действительно опираются на предыдущие (последние доступные) значения. И прогнозирование через `ARIMA` для заполнения пропусков у макроэкономических признаков, а также того самого `PRMB`, выделенного ранее.

С помощью `.ffill()` мы обработаем следующие признаки: `usdrub`, `palladium_rates`, `platinum_rates`, `silver_rates`, `gold_rates`, `IMOEX_close`, `IMOEX_volume`, `IMOEX_gap_hl`, `IMOEX_gap_co`, `S&P500_close`, `S&P500_gap_co`, `S&P500_gap_hl`, `S&P500_volume`, `effr`.

## Исследование курсов валюты и драг металлов

**Замечание:** если исследовать возможные причины пропусков в `usdrub`, `gold_rates`, `silver_rates`, `platinum_rates` и `palladium_rates`, то стоит обратить внимание, что в 205 из 311 понедельников этих курсов не было. Не уверен, но кажется, что прикол в том, что ЦБ в пн пользуется курсов за пт **(подтверждение не смог почему-то найти)**. Но на этот день недели есть явное смещение, еще одно оправдание для использования `fill forward`.

In [33]:
fintech[~(fintech['usdrub'].isnull()) & (fintech['day_of_week'] == 'Monday')].shape

(56, 255)

In [34]:
fintech[(fintech['usdrub'].isnull()) & (fintech['day_of_week'] == 'Monday')].shape

(205, 255)

In [35]:
fintech['usdrub'] = fintech['usdrub'].ffill()
fintech['gold_rates'] = fintech['gold_rates'].ffill()
fintech['silver_rates'] = fintech['silver_rates'].ffill()
fintech['platinum_rates'] = fintech['platinum_rates'].ffill()
fintech['palladium_rates'] = fintech['palladium_rates'].ffill()

In [36]:
fintech['gold_rates'].isna().sum()

0

## Исследование IMOEX и S&P500


In [37]:
fintech[fintech['IMOEX_volume'].isnull()]

,volume_NTRS_USA,close_NTRS_USA,volume_EWBC_USA,close_EWBC_USA,volume_CG_USA,close_CG_USA,volume_CBOE_USA,close_CBOE_USA,volume_BLK_USA,close_BLK_USA,...,gold_rates,silver_rates,platinum_rates,palladium_rates,is_workday,day_of_week,S&P500_gap_hl,S&P500_gap_co,IMOEX_gap_hl,IMOEX_gap_co
date,,,,,,,,,,,,,,,,,,,,,
2019-05-09,804008.0,96.97,748222.0,50.17,593700.0,20.76,999923.0,103.60,435546.0,464.22,...,3290.00,41.27,2069.61,3303.26,False,Thursday,4.0300,1.43,NaN,NaN
2019-06-12,2207877.0,88.01,1765096.0,43.50,281900.0,21.57,617279.0,107.70,430122.0,446.25,...,3025.36,35.14,1823.92,3816.07,False,Wednesday,1.4400,-0.25,NaN,NaN
2019-11-04,691773.0,103.87,1119061.0,45.43,976300.0,27.98,592013.0,113.52,551836.0,476.53,...,2715.75,31.73,1854.48,2884.75,False,Monday,1.0400,-0.48,NaN,NaN
2019-12-31,741362.0,106.24,1167529.0,48.70,1362600.0,32.08,384804.0,120.00,296863.0,502.70,...,3008.36,35.48,1896.77,3795.53,False,Tuesday,1.9800,1.33,NaN,NaN
2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3031.25,35.48,1932.59,3821.40,False,Wednesday,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23,1098178.0,81.48,667150.0,72.36,1864387.0,44.64,1121752.0,197.15,492606.0,813.59,...,6043.85,68.81,2689.80,2901.53,False,Friday,3.0300,-1.42,NaN,NaN
2024-03-08,1225456.0,81.81,1038029.0,76.82,2165911.0,47.35,818779.0,186.69,396217.0,836.12,...,6814.03,78.37,2805.70,2787.94,False,Friday,7.0901,-3.74,NaN,NaN
2024-05-01,1062849.0,83.05,758016.0,75.56,6031390.0,40.97,535367.0,179.90,614845.0,751.58,...,6872.79,80.33,2680.69,2994.67,False,Wednesday,8.3250,-1.03,NaN,NaN


У данных об `S&P500` не получилось выявить каких-то закономерностей и понять причины пропусков у данных о индексах. Поэтому просто заполним пропуски через `.ffill()`. В случае `IMOEX` есть особенный временной промежуток: март 2022 года, когда данные не обновлялись Мосбиржей. Так как нам закономерность отсутсвия данных кажется непонятной, то было принято решение обработать `IMOEX` тоже через `ARIMA`.

In [38]:
fintech['S&P500_close'] = fintech['S&P500_close'].ffill()
fintech['S&P500_volume'] = fintech['S&P500_volume'].ffill()
fintech['S&P500_gap_hl'] = fintech['S&P500_gap_hl'].ffill()
fintech['S&P500_gap_co'] = fintech['S&P500_gap_co'].ffill()

## Исследование EFFR

In [39]:
fintech['effr'].isna().sum()

6

Посмотрев, где находятся пропуски у этого признака, то увидим, что за последние даты есть все 6 Nans, что вполне естественно. В целом, можно как попрощаться с этим датами, так и заполнить их с помощью `.ffill()`

In [40]:
#fintech = fintech[fintech.index < '2024-05-01']

In [41]:
fintech['effr'] = fintech['effr'].ffill()

In [42]:
fintech.isna().sum().sum()

11460

## Заполнение пропусков через ARIMA